# Modelo de Suavización Exponencial Triple - Holt-Winters (TES)

Este notebook implementa el método de Suavización Exponencial Triple (Triple Exponential Smoothing - Holt-Winters) para predecir las ventas de dos productos, utilizando tres tipos de validación temporal:
- Walk-Forward Validation
- Rolling Window
- Expanding Window

Posteriormente, se utiliza Optimización Bayesiana para encontrar los mejores hiperparámetros y entrenar el modelo final para producción.


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
import optuna
warnings.filterwarnings('ignore')


In [2]:
# Cargar el dataset
df = pd.read_csv('data-set.csv', index_col=0)
print("Dataset cargado:")
print(f"Forma del dataset: {df.shape}")
print(f"\nPrimeras filas:")
print(df.head())
print(f"\nÚltimas filas:")
print(df.tail())
print(f"\nInformación del dataset:")
print(df.info())


Dataset cargado:
Forma del dataset: (127, 2)

Primeras filas:
    producto1   producto2
1  500.000000  200.000000
2  497.400893  210.686220
3  478.605317  222.018584
4  486.454125  233.920990
5  479.695678  238.402098

Últimas filas:
      producto1   producto2
123  164.610771  629.293034
124  150.881839  637.099467
125  151.788470  653.155282
126  137.047639  672.528345
127  141.990873  676.058092

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 1 to 127
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   producto1  127 non-null    float64
 1   producto2  127 non-null    float64
dtypes: float64(2)
memory usage: 3.0 KB
None


## Función Holt-Winters (Triple Exponential Smoothing)

Implementación del método de Suavización Exponencial Triple (Holt-Winters) para realizar predicciones. En Holt-Winters, se utilizan tres parámetros de suavización:
- alpha (α): controla el peso dado a las observaciones más recientes del nivel
- beta (β): controla el peso dado a las observaciones más recientes de la tendencia
- gamma (γ): controla el peso dado a las observaciones más recientes de la estacionalidad


In [3]:
def holt_winters_forecast(data, alpha=None, beta=None, gamma=None, seasonal_periods=12, forecast_horizon=1):
    """
    Calcula la Suavización Exponencial Triple (Holt-Winters) y realiza predicciones.
    
    Parameters:
    -----------
    data : array-like
        Serie temporal de datos
    alpha : float, optional
        Parámetro de suavización del nivel (0 < alpha <= 1). Si es None, se optimiza automáticamente.
    beta : float, optional
        Parámetro de suavización de la tendencia (0 < beta <= 1). Si es None, se optimiza automáticamente.
    gamma : float, optional
        Parámetro de suavización de la estacionalidad (0 < gamma <= 1). Si es None, se optimiza automáticamente.
    seasonal_periods : int
        Período estacional (por defecto 12 para datos mensuales con estacionalidad anual)
    forecast_horizon : int
        Número de períodos a predecir (por defecto 1)
    
    Returns:
    --------
    float or array
        Predicción(es) usando Holt-Winters
    """
    if len(data) < 2 * seasonal_periods:
        # Si no hay suficientes datos para estacionalidad, usar Holt (sin estacionalidad)
        data_series = pd.Series(data)
        try:
            model = ExponentialSmoothing(
                data_series,
                trend='add',
                seasonal=None,
                initialization_method='estimated'
            )
            if alpha is not None and beta is not None:
                fit = model.fit(smoothing_level=alpha, smoothing_trend=beta, optimized=False)
            else:
                fit = model.fit(optimized=True)
            forecast = fit.forecast(steps=forecast_horizon)
            if forecast_horizon == 1:
                return float(forecast.iloc[0]) if hasattr(forecast, 'iloc') else float(forecast[0])
            else:
                return forecast.values if hasattr(forecast, 'values') else np.array(forecast)
        except:
            return data[-1] if len(data) > 0 else 0
    
    # Convertir a pandas Series para usar con statsmodels
    data_series = pd.Series(data)
    
    try:
        # Crear y ajustar el modelo Holt-Winters
        # Triple Exponential Smoothing: trend='add', seasonal='add'
        model = ExponentialSmoothing(
            data_series,
            trend='add',
            seasonal='add',
            seasonal_periods=seasonal_periods,
            initialization_method='estimated'
        )
        
        # Si alpha, beta y gamma son None, dejar que el modelo los optimice
        # Si están especificados, usarlos
        if alpha is not None and beta is not None and gamma is not None:
            fit = model.fit(
                smoothing_level=alpha, 
                smoothing_trend=beta, 
                smoothing_seasonal=gamma, 
                optimized=False
            )
        else:
            fit = model.fit(optimized=True)
        
        # Realizar predicción
        forecast = fit.forecast(steps=forecast_horizon)
        
        # Si solo se predice un paso, devolver un escalar
        if forecast_horizon == 1:
            # Manejar tanto Series como array
            if hasattr(forecast, 'iloc'):
                return float(forecast.iloc[0])
            else:
                return float(forecast[0])
        else:
            # Devolver array de predicciones
            if hasattr(forecast, 'values'):
                return forecast.values
            else:
                return np.array(forecast)
        
    except Exception as e:
        # En caso de error, usar el último valor como predicción
        print(f"Advertencia en Holt-Winters: {e}")
        return data[-1]


## Función 1: Walk-Forward Validation
    
En Walk-Forward Validation, se entrena el modelo con datos hasta un punto específico y se valida con el siguiente punto. Luego se avanza un paso y se repite el proceso.


In [4]:
def walk_forward_validation(dataset, alpha=None, beta=None, gamma=None, seasonal_periods=12, train_size_min=24):
    """
    Realiza validación Walk-Forward para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    alpha : float, optional
        Parámetro de suavización del nivel para Holt-Winters. Si es None, se optimiza en cada ventana.
    beta : float, optional
        Parámetro de suavización de la tendencia para Holt-Winters. Si es None, se optimiza en cada ventana.
    gamma : float, optional
        Parámetro de suavización de la estacionalidad para Holt-Winters. Si es None, se optimiza en cada ventana.
    seasonal_periods : int
        Período estacional (por defecto 12)
    train_size_min : int
        Tamaño mínimo de entrenamiento antes de comenzar la validación (por defecto 24, mínimo 2*seasonal_periods)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Walk-Forward Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = holt_winters_forecast(
                    train_data, 
                    alpha=alpha, 
                    beta=beta, 
                    gamma=gamma, 
                    seasonal_periods=seasonal_periods,
                    forecast_horizon=1
                )
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


In [5]:
def rolling_window_validation(dataset, alpha=None, beta=None, gamma=None, seasonal_periods=12, train_window_size=24):
    """
    Realiza validación Rolling Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    alpha : float, optional
        Parámetro de suavización del nivel para Holt-Winters. Si es None, se optimiza en cada ventana.
    beta : float, optional
        Parámetro de suavización de la tendencia para Holt-Winters. Si es None, se optimiza en cada ventana.
    gamma : float, optional
        Parámetro de suavización de la estacionalidad para Holt-Winters. Si es None, se optimiza en cada ventana.
    seasonal_periods : int
        Período estacional (por defecto 12)
    train_window_size : int
        Tamaño fijo de la ventana de entrenamiento (por defecto 24, mínimo 2*seasonal_periods)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Rolling Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar desde train_window_size hasta n-1
        for i in range(train_window_size, n):
            # Datos de entrenamiento: ventana fija de tamaño train_window_size
            train_data = data[i - train_window_size:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = holt_winters_forecast(
                    train_data, 
                    alpha=alpha, 
                    beta=beta, 
                    gamma=gamma, 
                    seasonal_periods=seasonal_periods,
                    forecast_horizon=1
                )
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_window_size + 1,
                'ventana': f"{i - train_window_size}-{i}",
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_window_size + 1) % 20 == 0:
                print(f"  Iteración {i - train_window_size + 1}/{n - train_window_size}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 3: Expanding Window Validation

En Expanding Window Validation, la ventana de entrenamiento crece con cada iteración, comenzando desde un tamaño mínimo y expandiéndose hasta incluir todos los datos disponibles hasta ese punto.


In [6]:
def expanding_window_validation(dataset, alpha=None, beta=None, gamma=None, seasonal_periods=12, train_size_min=24):
    """
    Realiza validación Expanding Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    alpha : float, optional
        Parámetro de suavización del nivel para Holt-Winters. Si es None, se optimiza en cada ventana.
    beta : float, optional
        Parámetro de suavización de la tendencia para Holt-Winters. Si es None, se optimiza en cada ventana.
    gamma : float, optional
        Parámetro de suavización de la estacionalidad para Holt-Winters. Si es None, se optimiza en cada ventana.
    seasonal_periods : int
        Período estacional (por defecto 12)
    train_size_min : int
        Tamaño mínimo inicial de la ventana de entrenamiento (por defecto 24, mínimo 2*seasonal_periods)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Expanding Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i (ventana que se expande)
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = holt_winters_forecast(
                    train_data, 
                    alpha=alpha, 
                    beta=beta, 
                    gamma=gamma, 
                    seasonal_periods=seasonal_periods,
                    forecast_horizon=1
                )
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,  # Tamaño de la ventana que se expande
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Ejecución de las Validaciones

Después de definir las funciones que permitirán ejecutar el modelo teniendo en cuenta los tres tipos de validación, ahora ejecutamos las tres validaciones para cada producto y calculamos las métricas promedio.


In [7]:
# Parámetros de validación
SEASONAL_PERIODS = 12  # Período estacional (asumiendo datos mensuales con estacionalidad anual)
TRAIN_SIZE_MIN = 24  # Tamaño mínimo de entrenamiento (2 * seasonal_periods)
TRAIN_WINDOW_SIZE = 24  # Tamaño fijo para Rolling Window

# Ejecutar Walk-Forward Validation
print("=" * 60)
print("WALK-FORWARD VALIDATION")
print("=" * 60)
results_wf = walk_forward_validation(
    df, 
    alpha=None, 
    beta=None, 
    gamma=None, 
    seasonal_periods=SEASONAL_PERIODS,
    train_size_min=TRAIN_SIZE_MIN
)


WALK-FORWARD VALIDATION

=== Walk-Forward Validation para producto1 ===
  Iteración 20/103: RMSE=0.9233, MAE=0.9233
  Iteración 40/103: RMSE=1.8486, MAE=1.8486
  Iteración 60/103: RMSE=14.3306, MAE=14.3306
  Iteración 80/103: RMSE=3.9068, MAE=3.9068
  Iteración 100/103: RMSE=10.2808, MAE=10.2808
  Total de iteraciones: 103

=== Walk-Forward Validation para producto2 ===
  Iteración 20/103: RMSE=25.0052, MAE=25.0052
  Iteración 40/103: RMSE=10.4836, MAE=10.4836
  Iteración 60/103: RMSE=3.3725, MAE=3.3725
  Iteración 80/103: RMSE=21.0236, MAE=21.0236
  Iteración 100/103: RMSE=6.1529, MAE=6.1529
  Total de iteraciones: 103


In [8]:
# Ejecutar Rolling Window Validation
print("\n" + "=" * 60)
print("ROLLING WINDOW VALIDATION")
print("=" * 60)
results_rw = rolling_window_validation(
    df, 
    alpha=None, 
    beta=None, 
    gamma=None, 
    seasonal_periods=SEASONAL_PERIODS,
    train_window_size=TRAIN_WINDOW_SIZE
)



ROLLING WINDOW VALIDATION

=== Rolling Window Validation para producto1 ===
  Iteración 20/103: RMSE=7.7766, MAE=7.7766
  Iteración 40/103: RMSE=11.9144, MAE=11.9144
  Iteración 60/103: RMSE=16.4254, MAE=16.4254
  Iteración 80/103: RMSE=6.5048, MAE=6.5048
  Iteración 100/103: RMSE=3.4189, MAE=3.4189
  Total de iteraciones: 103

=== Rolling Window Validation para producto2 ===
  Iteración 20/103: RMSE=12.4031, MAE=12.4031
  Iteración 40/103: RMSE=7.7449, MAE=7.7449
  Iteración 60/103: RMSE=10.7454, MAE=10.7454
  Iteración 80/103: RMSE=19.3286, MAE=19.3286
  Iteración 100/103: RMSE=8.5733, MAE=8.5733
  Total de iteraciones: 103


In [9]:
# Ejecutar Expanding Window Validation
print("\n" + "=" * 60)
print("EXPANDING WINDOW VALIDATION")
print("=" * 60)
results_ew = expanding_window_validation(
    df, 
    alpha=None, 
    beta=None, 
    gamma=None, 
    seasonal_periods=SEASONAL_PERIODS,
    train_size_min=TRAIN_SIZE_MIN
)



EXPANDING WINDOW VALIDATION

=== Expanding Window Validation para producto1 ===
  Iteración 20/103: RMSE=0.9244, MAE=0.9244
  Iteración 40/103: RMSE=1.8477, MAE=1.8477
  Iteración 60/103: RMSE=14.3280, MAE=14.3280
  Iteración 80/103: RMSE=3.9045, MAE=3.9045
  Iteración 100/103: RMSE=10.2808, MAE=10.2808
  Total de iteraciones: 103

=== Expanding Window Validation para producto2 ===
  Iteración 20/103: RMSE=25.0017, MAE=25.0017
  Iteración 40/103: RMSE=10.4845, MAE=10.4845
  Iteración 60/103: RMSE=3.3725, MAE=3.3725
  Iteración 80/103: RMSE=21.0230, MAE=21.0230
  Iteración 100/103: RMSE=6.1414, MAE=6.1414
  Total de iteraciones: 103


In [10]:
def calcular_metricas_promedio(results, nombre_validacion):
    """
    Calcula las métricas promedio para cada producto.
    
    Parameters:
    -----------
    results : dict
        Resultados de la validación
    nombre_validacion : str
        Nombre del tipo de validación
    
    Returns:
    --------
    dict
        Diccionario con métricas promedio por producto
    """
    metricas_promedio = {}
    
    print(f"\n{'='*60}")
    print(f"MÉTRICAS PROMEDIO - {nombre_validacion.upper()}")
    print(f"{'='*60}")
    
    for producto in results.keys():
        metrics_list = results[producto]
        
        # Calcular promedios
        rmse_promedio = np.mean([m['rmse'] for m in metrics_list])
        mae_promedio = np.mean([m['mae'] for m in metrics_list])
        
        metricas_promedio[producto] = {
            'rmse_promedio': rmse_promedio,
            'mae_promedio': mae_promedio,
            'num_iteraciones': len(metrics_list)
        }
        
        print(f"\n{producto}:")
        print(f"  RMSE Promedio: {rmse_promedio:.4f}")
        print(f"  MAE Promedio: {mae_promedio:.4f}")
        print(f"  Número de iteraciones: {len(metrics_list)}")
    
    return metricas_promedio

# Calcular métricas promedio para cada tipo de validación
metricas_wf = calcular_metricas_promedio(results_wf, "Walk-Forward")
metricas_rw = calcular_metricas_promedio(results_rw, "Rolling Window")
metricas_ew = calcular_metricas_promedio(results_ew, "Expanding Window")



MÉTRICAS PROMEDIO - WALK-FORWARD

producto1:
  RMSE Promedio: 7.6618
  MAE Promedio: 7.6618
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 12.1809
  MAE Promedio: 12.1809
  Número de iteraciones: 103

MÉTRICAS PROMEDIO - ROLLING WINDOW

producto1:
  RMSE Promedio: 9.3012
  MAE Promedio: 9.3012
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 14.2423
  MAE Promedio: 14.2423
  Número de iteraciones: 103

MÉTRICAS PROMEDIO - EXPANDING WINDOW

producto1:
  RMSE Promedio: 7.6617
  MAE Promedio: 7.6617
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 12.1905
  MAE Promedio: 12.1905
  Número de iteraciones: 103


## Comparación de Resultados y Selección del Mejor Método de Validación

Comparamos los RMSE promedio de cada tipo de validación para seleccionar el mejor método.


In [11]:
# Crear DataFrame comparativo
comparacion = []

for producto in df.columns:
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Walk-Forward',
        'RMSE_Promedio': metricas_wf[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_wf[producto]['mae_promedio']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Rolling Window',
        'RMSE_Promedio': metricas_rw[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_rw[producto]['mae_promedio']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Expanding Window',
        'RMSE_Promedio': metricas_ew[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_ew[producto]['mae_promedio']
    })

df_comparacion = pd.DataFrame(comparacion)
print("\n" + "=" * 80)
print("COMPARACIÓN DE MÉTODOS DE VALIDACIÓN")
print("=" * 80)
print(df_comparacion.to_string(index=False))

# Identificar el mejor método para cada producto
print("\n" + "=" * 80)
print("MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)")
print("=" * 80)
for producto in df.columns:
    producto_df = df_comparacion[df_comparacion['Producto'] == producto]
    mejor = producto_df.loc[producto_df['RMSE_Promedio'].idxmin()]
    print(f"\n{producto}:")
    print(f"  Mejor método: {mejor['Validacion']}")
    print(f"  RMSE Promedio: {mejor['RMSE_Promedio']:.4f}")



COMPARACIÓN DE MÉTODOS DE VALIDACIÓN
 Producto       Validacion  RMSE_Promedio  MAE_Promedio
producto1     Walk-Forward       7.661766      7.661766
producto1   Rolling Window       9.301167      9.301167
producto1 Expanding Window       7.661674      7.661674
producto2     Walk-Forward      12.180917     12.180917
producto2   Rolling Window      14.242317     14.242317
producto2 Expanding Window      12.190520     12.190520

MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)

producto1:
  Mejor método: Expanding Window
  RMSE Promedio: 7.6617

producto2:
  Mejor método: Walk-Forward
  RMSE Promedio: 12.1809


## Optimización Bayesiana para Encontrar los Mejores Parámetros (Alpha, Beta y Gamma)

Utilizamos Optimización Bayesiana (Optuna) para encontrar los mejores parámetros alpha, beta y gamma para cada producto, usando el mejor método de validación identificado.


In [12]:
def objective_walk_forward(trial, data, seasonal_periods=12, train_size_min=24):
    """
    Función objetivo para optimización bayesiana usando Walk-Forward Validation.
    """
    alpha = trial.suggest_float('alpha', 0.01, 0.99, log=False)
    beta = trial.suggest_float('beta', 0.01, 0.99, log=False)
    gamma = trial.suggest_float('gamma', 0.01, 0.99, log=False)
    
    n = len(data)
    errors = []
    
    for i in range(train_size_min, n):
        train_data = data[:i]
        actual = data[i]
        
        try:
            prediction = holt_winters_forecast(
                train_data, 
                alpha=alpha, 
                beta=beta, 
                gamma=gamma, 
                seasonal_periods=seasonal_periods,
                forecast_horizon=1
            )
            error = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error):
                errors.append(error)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)

def objective_rolling_window(trial, data, seasonal_periods=12, train_window_size=24):
    """
    Función objetivo para optimización bayesiana usando Rolling Window Validation.
    """
    alpha = trial.suggest_float('alpha', 0.01, 0.99, log=False)
    beta = trial.suggest_float('beta', 0.01, 0.99, log=False)
    gamma = trial.suggest_float('gamma', 0.01, 0.99, log=False)
    
    n = len(data)
    errors = []
    
    for i in range(train_window_size, n):
        train_data = data[i - train_window_size:i]
        actual = data[i]
        
        try:
            prediction = holt_winters_forecast(
                train_data, 
                alpha=alpha, 
                beta=beta, 
                gamma=gamma, 
                seasonal_periods=seasonal_periods,
                forecast_horizon=1
            )
            error = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error):
                errors.append(error)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)

def objective_expanding_window(trial, data, seasonal_periods=12, train_size_min=24):
    """
    Función objetivo para optimización bayesiana usando Expanding Window Validation.
    """
    alpha = trial.suggest_float('alpha', 0.01, 0.99, log=False)
    beta = trial.suggest_float('beta', 0.01, 0.99, log=False)
    gamma = trial.suggest_float('gamma', 0.01, 0.99, log=False)
    
    n = len(data)
    errors = []
    
    for i in range(train_size_min, n):
        train_data = data[:i]
        actual = data[i]
        
        try:
            prediction = holt_winters_forecast(
                train_data, 
                alpha=alpha, 
                beta=beta, 
                gamma=gamma, 
                seasonal_periods=seasonal_periods,
                forecast_horizon=1
            )
            error = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error):
                errors.append(error)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)


In [13]:
# Determinar el mejor método de validación para cada producto
mejor_metodo_por_producto = {}

for producto in df.columns:
    producto_df = df_comparacion[df_comparacion['Producto'] == producto]
    mejor_idx = producto_df['RMSE_Promedio'].idxmin()
    mejor_metodo = producto_df.loc[mejor_idx, 'Validacion']
    mejor_metodo_por_producto[producto] = mejor_metodo
    print(f"{producto}: Mejor método = {mejor_metodo}")

# Realizar optimización bayesiana para cada producto usando su mejor método
mejores_params = {}

print("\n" + "=" * 80)
print("OPTIMIZACIÓN BAYESIANA")
print("=" * 80)

for producto in df.columns:
    print(f"\n--- Optimizando {producto} ---")
    data = df[producto].values
    mejor_metodo = mejor_metodo_por_producto[producto]
    
    # Crear función objetivo según el mejor método
    if mejor_metodo == 'Walk-Forward':
        objective_func = lambda trial: objective_walk_forward(trial, data, SEASONAL_PERIODS, TRAIN_SIZE_MIN)
    elif mejor_metodo == 'Rolling Window':
        objective_func = lambda trial: objective_rolling_window(trial, data, SEASONAL_PERIODS, TRAIN_WINDOW_SIZE)
    else:  # Expanding Window
        objective_func = lambda trial: objective_expanding_window(trial, data, SEASONAL_PERIODS, TRAIN_SIZE_MIN)
    
    # Crear estudio de Optuna
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
    
    # Ejecutar optimización
    study.optimize(objective_func, n_trials=50, show_progress_bar=True)
    
    mejores_params[producto] = {
        'alpha': study.best_params['alpha'],
        'beta': study.best_params['beta'],
        'gamma': study.best_params['gamma'],
        'best_rmse': study.best_value,
        'metodo_validacion': mejor_metodo
    }
    
    print(f"  Mejor alpha: {study.best_params['alpha']:.4f}")
    print(f"  Mejor beta: {study.best_params['beta']:.4f}")
    print(f"  Mejor gamma: {study.best_params['gamma']:.4f}")
    print(f"  Mejor RMSE: {study.best_value:.4f}")
    print(f"  Método usado: {mejor_metodo}")

print("\n" + "=" * 80)
print("RESUMEN DE MEJORES PARÁMETROS")
print("=" * 80)
for producto, params in mejores_params.items():
    print(f"\n{producto}:")
    print(f"  Alpha óptimo: {params['alpha']:.4f}")
    print(f"  Beta óptimo: {params['beta']:.4f}")
    print(f"  Gamma óptimo: {params['gamma']:.4f}")
    print(f"  RMSE: {params['best_rmse']:.4f}")
    print(f"  Método de validación: {params['metodo_validacion']}")


[I 2025-11-30 13:22:50,360] A new study created in memory with name: no-name-269a4e00-2f2d-4359-89c9-1f47d7fc292c


producto1: Mejor método = Expanding Window
producto2: Mejor método = Walk-Forward

OPTIMIZACIÓN BAYESIANA

--- Optimizando producto1 ---


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-11-30 13:22:50,772] Trial 0 finished with value: 43.941488661786245 and parameters: {'alpha': 0.3006395079576752, 'beta': 0.49055301901491244, 'gamma': 0.6412723028166583}. Best is trial 0 with value: 43.941488661786245.
[I 2025-11-30 13:22:51,412] Trial 1 finished with value: 17.761963766043998 and parameters: {'alpha': 0.19737127563035484, 'beta': 0.543512917012553, 'gamma': 0.27080047872517704}. Best is trial 1 with value: 17.761963766043998.
[I 2025-11-30 13:22:51,763] Trial 2 finished with value: 12.125918886381939 and parameters: {'alpha': 0.8861760930306928, 'beta': 0.5620350752734634, 'gamma': 0.45394729805613326}. Best is trial 2 with value: 12.125918886381939.
[I 2025-11-30 13:22:52,078] Trial 3 finished with value: 8.313944162510213 and parameters: {'alpha': 0.8446002289528575, 'beta': 0.15885262200906966, 'gamma': 0.2538208372487235}. Best is trial 3 with value: 8.313944162510213.
[I 2025-11-30 13:22:52,436] Trial 4 finished with value: 39.39535990888402 and paramet

[I 2025-11-30 13:23:08,592] A new study created in memory with name: no-name-46f15122-df29-440d-aafa-1c3c18e0a425


[I 2025-11-30 13:23:08,589] Trial 49 finished with value: 7.644573631962163 and parameters: {'alpha': 0.46792835720365655, 'beta': 0.8327436632152738, 'gamma': 0.041607326585322096}. Best is trial 45 with value: 7.265357662045609.
  Mejor alpha: 0.6675
  Mejor beta: 0.7319
  Mejor gamma: 0.0556
  Mejor RMSE: 7.2654
  Método usado: Expanding Window

--- Optimizando producto2 ---


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-11-30 13:23:08,962] Trial 0 finished with value: 14.062173598697493 and parameters: {'alpha': 0.7158523094323204, 'beta': 0.6488351031011607, 'gamma': 0.40414055725973186}. Best is trial 0 with value: 14.062173598697493.
[I 2025-11-30 13:23:09,269] Trial 1 finished with value: 172.01706881636684 and parameters: {'alpha': 0.7281517473628718, 'beta': 0.845380561862319, 'gamma': 0.8393515158537582}. Best is trial 0 with value: 14.062173598697493.
[I 2025-11-30 13:23:09,613] Trial 2 finished with value: 21.470809890879167 and parameters: {'alpha': 0.5088514559528106, 'beta': 0.10001071574430423, 'gamma': 0.7748579767572388}. Best is trial 0 with value: 14.062173598697493.
[I 2025-11-30 13:23:09,939] Trial 3 finished with value: 37.051851473411965 and parameters: {'alpha': 0.7119693465383556, 'beta': 0.2787768171583865, 'gamma': 0.9852271655621254}. Best is trial 0 with value: 14.062173598697493.
[I 2025-11-30 13:23:10,253] Trial 4 finished with value: 41.81462064588898 and paramete

## Modelo Final para Producción

Entrenamos el modelo final con los mejores parámetros encontrados usando todos los datos disponibles y realizamos predicciones para producción (1 pronóstico para el siguiente mes).


In [14]:
# Entrenar modelos finales para cada producto con los mejores parámetros
modelos_finales = {}
predicciones_finales = {}

print("=" * 80)
print("ENTRENAMIENTO DE MODELOS FINALES PARA PRODUCCIÓN")
print("=" * 80)

for producto in df.columns:
    print(f"\n--- {producto} ---")
    data = df[producto].values
    alpha_optimo = mejores_params[producto]['alpha']
    beta_optimo = mejores_params[producto]['beta']
    gamma_optimo = mejores_params[producto]['gamma']
    
    # Entrenar modelo final con todos los datos
    data_series = pd.Series(data)
    
    # Verificar si hay suficientes datos para estacionalidad
    if len(data) >= 2 * SEASONAL_PERIODS:
        model = ExponentialSmoothing(
            data_series,
            trend='add',
            seasonal='add',
            seasonal_periods=SEASONAL_PERIODS,
            initialization_method='estimated'
        )
        fit = model.fit(
            smoothing_level=alpha_optimo, 
            smoothing_trend=beta_optimo, 
            smoothing_seasonal=gamma_optimo, 
            optimized=False
        )
    else:
        # Si no hay suficientes datos, usar Holt (sin estacionalidad)
        print(f"  Advertencia: Datos insuficientes para estacionalidad. Usando Holt (sin estacionalidad).")
        model = ExponentialSmoothing(
            data_series,
            trend='add',
            seasonal=None,
            initialization_method='estimated'
        )
        fit = model.fit(
            smoothing_level=alpha_optimo, 
            smoothing_trend=beta_optimo, 
            optimized=False
        )
    
    modelos_finales[producto] = fit
    
    # Realizar predicción para el próximo mes (1 solo pronóstico)
    forecast_horizon = 1
    forecast = fit.forecast(steps=forecast_horizon)
    
    # Extraer el valor de la predicción (manejar tanto Series como array)
    if hasattr(forecast, 'iloc'):
        forecast_value = float(forecast.iloc[0])
    elif hasattr(forecast, 'values'):
        forecast_value = float(forecast.values[0])
    else:
        forecast_value = float(forecast[0])
    
    predicciones_finales[producto] = forecast_value
    
    print(f"  Alpha usado: {alpha_optimo:.4f}")
    print(f"  Beta usado: {beta_optimo:.4f}")
    print(f"  Gamma usado: {gamma_optimo:.4f}")
    print(f"  Último valor observado: {data[-1]:.4f}")
    print(f"  Predicción para el próximo mes: {forecast_value:.4f}")

# Crear DataFrame con las predicciones
df_predicciones = pd.DataFrame([predicciones_finales])
df_predicciones.index = ['Siguiente_Mes']

print("\n" + "=" * 80)
print("PREDICCIONES FINALES PARA PRODUCCIÓN")
print("=" * 80)
print(df_predicciones)


ENTRENAMIENTO DE MODELOS FINALES PARA PRODUCCIÓN

--- producto1 ---
  Alpha usado: 0.6675
  Beta usado: 0.7319
  Gamma usado: 0.0556
  Último valor observado: 141.9909
  Predicción para el próximo mes: 139.0054

--- producto2 ---
  Alpha usado: 0.6068
  Beta usado: 0.5995
  Gamma usado: 0.2718
  Último valor observado: 676.0581
  Predicción para el próximo mes: 680.4668

PREDICCIONES FINALES PARA PRODUCCIÓN
                producto1   producto2
Siguiente_Mes  139.005388  680.466756


## Resumen Final

### Resultados de Validación

- **Walk-Forward Validation**: Se evaluó el modelo avanzando paso a paso en el tiempo
- **Rolling Window Validation**: Se evaluó con una ventana fija que se desplaza
- **Expanding Window Validation**: Se evaluó con una ventana que crece progresivamente

### Optimización Bayesiana

Se utilizó Optimización Bayesiana (Optuna) para encontrar los mejores parámetros alpha, beta y gamma para cada producto, usando el método de validación que obtuvo el menor RMSE promedio.

### Modelo Final

Los modelos finales fueron entrenados con todos los datos disponibles usando los mejores parámetros encontrados (alpha, beta y gamma), y están listos para realizar predicciones en producción para el siguiente mes.
